Задание 1: Анализ финансовых транзакций и клиентов.

Описание:

В тестовом задании вам предоставляются два набора данных:
~~~~~~
Файл transactions_data.xlsx (данные о финансовых транзакциях)

•	transaction_id – уникальный идентификатор транзакции
•	client_id – идентификатор клиента, совершившего транзакцию
•	transaction_date – дата проведения транзакции
•	service – тип оказанной финансовой услуги
•	amount – сумма транзакции (выручка)
•	payment_method – способ оплаты (банковский перевод, кредитная карта и т. д.)
•	city – город, в котором была совершена транзакция
•	consultant – имя финансового консультанта, оформившего транзакцию

Файл clients_data.json (данные о клиентах):

•	client_id – уникальный идентификатор клиента
•	age – возраст клиента
•	gender – пол клиента
•	net_worth – чистая стоимость активов клиента

Данные могут содержать пропуски, ошибки (например, некорректные даты), а также аномальные значения (отрицательные суммы, несуществующие услуги и т. д.), поэтому перед анализом необходимо их очистить и нормализовать.

Задачи:

1.	Очистка и подготовка данных. Проверить данные на пропущенные и аномальные значения (например, отрицательные суммы транзакций). Удалить или исправить некорректные данные. Привести формат дат к единому стандарту.

In [3]:
import pandas as pd

In [4]:
#Чтение файлов:
df1T = pd.read_excel('Источники/Транзакции/transactions_data.xlsx')
df2C = pd.read_json('Источники/Клиенты/clients_data.json')  

In [5]:
#Проверки на дубли:
df1T.duplicated().sum()
df2C.duplicated().sum()

np.int64(0)

In [6]:
#Проверка на пустые значения:
df1T.isna().sum() #Проверка на Пустые значения в Транзакциях. 

#UPD1: Значения по 0 должно быть, после очистки. 
#UPD2: Есть данные где остальные присутствуют, но нет дат. Заполенено "INVALID_DATE".
#UPD3: Удалил данные с заменной INVALID_DATE на Nan. 

transaction_id      0
client_id           0
transaction_date    0
service             0
amount              0
payment_method      0
city                0
consultant          0
dtype: int64

In [7]:
#Проверка на пустые значения:
df2C.isna().sum() #Проверка на Пустые значения в Клиентах.

id              0
age             0
gender       2863
net_worth       0
dtype: int64

План очистки данных файла Транзакции:

Удалю данные: 
~~~
-Где нет id транзакции. Нет указаний, в тестовом задании, что с ним делать. И пока нет возможности спросить как откуда до грузить id. 
-Где нет id клиента в файле транзакции. Тоже самое.
-Где есть поле "Неизвестная услуга". Тоже самое. 
-Где нет в поле "transaction_date" даты. Пока под вопросомю.

UPD1: И ещё есть формат где даты нет, но остальные данные на месте. По итогу оставили данные где нет дат, но остальные данные есть. 

UPD2:Пока не трогаю данные где есть "Неизвестный консультант" и city - "Неизвестный город"

UPD3:Удалил данные где пропуск дат, возможно буду мешать прогнозированию. 

In [8]:
#Очистка данных на пустые значения файла Транзакции:
df1T = df1T.dropna() 

#Провоерка выше.

In [9]:
#Сохранение данных:
df1T.to_excel('Источники/Транзакции/transactions_data.xlsx', index=False)

In [10]:
#Замена INVALID_DATE на Nan

df1T['transaction_date'] = pd.to_datetime(df1T['transaction_date'], errors='coerce')

#Дает ошибку, если за ранее не изменнить.
#"Unknown datetime string format, unable to parse: INVALID_DATE, at position 15".

In [11]:
#Преобразование дат к одному в формату:
df1T['transaction_date'] = pd.to_datetime(df1T['transaction_date'])
#Приведение всех дат к одному формату с точностью до секунд:
df1T['transaction_date'] = df1T['transaction_date'].dt.strftime('%Y-%m-%d %H:%M:%S')

In [12]:
#Сохранение данных:
df1T.to_excel('Источники/Транзакции/transactions_data.xlsx', index=False)

Есть файлы где нет даты, но остальные в норме. Продолжим удаление, но по условиям. 
*Открыл вручуную файл Excel.*. 

In [13]:
#Очистка данных на пустые значения файла Транзакции:
df1T = df1T.dropna() 

#Пересмотрел план, удаляю данные где нет даты. Так как, возможно, будет мешать прогнозированию.

In [14]:
#Удаление строк с "Неизвестной услугой":
df1T = df1T[df1T['service'] != 'Неизвестная услуга']

In [15]:
#Сохранение данных:
df1T.to_excel('Источники/Транзакции/transactions_data.xlsx', index=False)

In [16]:
#Продолжаем очистку, теперь зайдем со стороны аномальных значений:
#Использую "Межквартильный размах" (IQR). 

# Рассчитываем первый и третий квартели, чтобы посчитать IQR.
Q1 = df1T['amount'].quantile(0.25)
Q3 = df1T['amount'].quantile(0.75)

#Расчитываем IQR:
IQR = Q3 - Q1

#Устанавливаем границы 
lower_bound = max(0, Q1 - 1.5 * IQR) #Нижняя. Нижняя не может быть меньше 0. (...отрицательные суммы).
upper_bound = Q3 + 1.5 * IQR #Верхняя.

#Вывод границ: 
print(lower_bound)
print(upper_bound)

0
145183.82273751637


In [17]:
#Вывод аномальных значений из файла:
anomal = df1T[(df1T['amount'] < lower_bound) | (df1T['amount'] > upper_bound)]
print(anomal[['service','amount']])

                              service         amount
21            Финансовое планирование  147439.369086
411   Инвестиционное консультирование  145713.605919
445   Инвестиционное консультирование  149546.564423
454            Налоговое планирование  149407.108459
569   Инвестиционное консультирование  146218.600215
903            Налоговое планирование  147333.389591
1524  Инвестиционное консультирование  148549.499778
2782              Управление активами  145912.853299
2830          Финансовое планирование  147667.516120
2938          Финансовое планирование  148470.649643
3084          Финансовое планирование  147064.285766
3539        Структурирование капитала  145827.584354
4109  Инвестиционное консультирование  146821.816904
5030           Налоговое планирование  146609.123497
5487  Инвестиционное консультирование  148873.575903
5599  Инвестиционное консультирование  147194.185379
5829  Инвестиционное консультирование  147704.029797
6338  Инвестиционное консультирование  145955.

In [18]:
#Удаление аномальных значений: 
df1T = df1T[(df1T['amount'] >= lower_bound) & (df1T['amount'] <= upper_bound)] 

In [19]:
#Сохранение данных:
df1T.to_excel('Источники/Транзакции/transactions_data.xlsx', index=False)

In [20]:
#Проверка: 
#Вывод аномальных значений из файла Транзакций:
anomal = df1T[(df1T['amount'] < lower_bound) | (df1T['amount'] > upper_bound)]

#Вывод:
print(anomal)

Empty DataFrame
Columns: [transaction_id, client_id, transaction_date, service, amount, payment_method, city, consultant]
Index: []


План очистки данных файла Клиентов:
~~~
-Удалить строки где нет id.
-Удалить строки где нет Общей суммы активов. 
-Удалить строки где нет возраста. 

*Пока не удаляю данные где нет пола, так как в задаче, вижу особо он не фигурирует, и может не влиять особо на анализ. Возможно в процессе что-то поменяю. 

In [21]:
#Очистка данных на пустые значения файла Клиентов, где нет id:
df2C = df2C.dropna(subset=['id'])

In [22]:
#Очистка данных на пустые значения файла Клиентов, где нет общей суммы активов:
df2C = df2C.dropna(subset=['net_worth'])

In [23]:
#Очистка данных на пустые значения файла Клиентов, где нет возраста:
df2C = df2C.dropna(subset=['age'])

In [24]:
#Сохранение данных:
df2C.to_json('Источники/Клиенты/clients_data.json', index=False)

In [25]:
#Проверка на аномальные значения капитала, отрицательные значения:
munuscap = df2C[df2C['net_worth'] < 0] #Проверим сначала, есть ли отрицательные значения. 
print(munuscap[['id', 'net_worth']])

Empty DataFrame
Columns: [id, net_worth]
Index: []


In [26]:
#Теперь зайдем со стороны "Межквартильный размах" (IQR):

# Рассчитываем первый и третий квартели, чтобы посчитать IQR.
Q1C = df2C['net_worth'].quantile(0.25)
Q3C = df2C['net_worth'].quantile(0.75)

#Расчитываем IQR:
IQR_C = Q3C - Q1C

#Устанавливаем границы 
lower_boundC = max(0, Q1C - 1.5 * IQR_C) #Нижняя. Нижняя не может быть меньше 0. (...отрицательные суммы).
upper_boundC = Q3C + 1.5 * IQR_C #Верхняя.

#Вывод границ: 
print(lower_boundC)
print(upper_boundC)


0
7400150.275


In [27]:
#Проверка: 
#Вывод аномальных значений из файла Клиентов:
anomalC = df2C[(df2C['net_worth'] < lower_boundC) | (df2C['net_worth'] > upper_boundC)]

#Вывод:
print(anomalC)

Empty DataFrame
Columns: [id, age, gender, net_worth]
Index: []


In [28]:
#Сохранение данных:
df2C.to_json('Источники/Клиенты/clients_data.json', index=False)

Закончили очистку. Далее. 

Задачи:  

2.  Анализ данных.
~~~
-Определить топ-5 наиболее популярных услуг по количеству заказов. 
-Рассчитать среднюю сумму транзакций по каждому городу. 
-Определить услугу с наибольшей выручкой (по сумме amount). 
-Вычислить процент транзакций по способам оплаты (наличные, банковский перевод и т. д.). 
-Рассчитать выручку за последний месяц (по сумме amount).

In [51]:
#Топ-5 наиболее популярных услуг по количеству заказов:
TOP_5 = df1T.groupby('service').size().nlargest(5) 
print(TOP_5)
#.groupby - группировка. 
#.nlargest - возвращает самые большие значения из серии. 
#.size подсчет по колонке. 

service
Инвестиционное консультирование    1912
Управление активами                1569
Финансовое планирование            1213
Налоговое планирование             1186
Структурирование капитала          1181
dtype: int64
